In [ ]:
import requests
from google.colab import drive
import csv
import os
import time

In [ ]:
#Mount Google drive, provide auth code to write/read from G drive. You don't need to mount this if running this notebook on local.
drive.mount("/drive")

Mounted at /drive


In [ ]:
# api-endpoint
ISSUE_URL = "https://api.github.com/repos/{}/issues" 

repo_list = [
             "OpenGenus/cosmos","playframework/playframework","rundeck/rundeck","ivanperez-keera/Yampa","andmorefine/since-co","sbt/sbt-aspectj",
             "ComputerScienceHouse/Constitution","sbt/sbt-findbugs","oceanprotocol/contracts","contentful/contentful-management.rb","okta/samples-js-react-native"
            ]

issues_dict = {}
repo_name= "OpenGenus/cosmos"
for repo_name in repo_list:
    r = requests.get(url = ISSUE_URL.format(repo_name))

    # extracting data in json format
    issues = r.json()
    top_issue_url = issues[0]["url"] #finding the first result which is the most recent issue to iterate over number of issues
    print(top_issue_url)

    number_of_issues = top_issue_url.split("/")[-1]
    print(number_of_issues)

    issues_dict[repo_name] =number_of_issues
print(issues_dict)

https://api.github.com/repos/OpenGenus/cosmos/issues/6260
6260
https://api.github.com/repos/playframework/playframework/issues/10999
10999
https://api.github.com/repos/rundeck/rundeck/issues/7328
7328
https://api.github.com/repos/ivanperez-keera/Yampa/issues/194
194
https://api.github.com/repos/andmorefine/since-co/issues/1367
1367
https://api.github.com/repos/sbt/sbt-aspectj/issues/49
49
https://api.github.com/repos/ComputerScienceHouse/Constitution/issues/199
199
https://api.github.com/repos/sbt/sbt-findbugs/issues/27
27
https://api.github.com/repos/oceanprotocol/contracts/issues/367
367
https://api.github.com/repos/contentful/contentful-management.rb/issues/232
232
https://api.github.com/repos/okta/samples-js-react-native/issues/102
102
{'OpenGenus/cosmos': '6260', 'playframework/playframework': '10999', 'rundeck/rundeck': '7328', 'ivanperez-keera/Yampa': '194', 'andmorefine/since-co': '1367', 'sbt/sbt-aspectj': '49', 'ComputerScienceHouse/Constitution': '199', 'sbt/sbt-findbugs': '

In [ ]:
fileop = open(filename,'r')
print(fileop.readlines())
fileop.close()

['Repo,Issue-no,Issue-title,Issue-body,Issue-reaction-total-count,Issue-reaction-plus-one,Issue-reaction-minus-one,Issue-comments\n', 'contentful/contentful-management.rb,232,Move Travis CI to travis-ci.com,"Transfer [one on travis-ci.org](https:  travis-ci.org github contentful contentful-management.rb) to [app.travis-ci.com](https:  app.travis-ci.com github contentful contentful-management.rb) like [other repos on Contentful](https:  app.travis-ci.com github contentful) since .org was shut down on June 15th, 2021.\n', '\n', 'Signed-off-by: Takuya Noguchi <takninnovationresearch@gmail.com>",0,0,0,1\n', 'contentful/contentful-management.rb,231,Adds getter setter for EditorInterface#sidebar property,Fixes #229,0,0,0,4\n', 'contentful/contentful-management.rb,230,Tasks implementation missing in contentful-management.rb,"Need to be able to create read update delete Tasks on an entry via contentful-management.rb.\n', '\n', 'need to:\n', '   - create a Task Resource\n', '   - include it in 

In [ ]:
search_file_url ="https://api.github.com/repos/{}/issues/{}"
repos_dict = {'OpenGenus/cosmos': '6260', 'playframework/playframework': '10999', 'rundeck/rundeck': '7328', 
              'ivanperez-keera/Yampa': '194', 'andmorefine/since-co': '1367', 'sbt/sbt-aspectj': '49', 
              'ComputerScienceHouse/Constitution': '199', 'sbt/sbt-findbugs': '27', 'oceanprotocol/contracts': '367', 
              'contentful/contentful-management.rb': '232', 'okta/samples-js-react-native': '102'
              }

git_token = 'token'
headers = {'Authorization': 'token %s' % git_token}
sleep_time = 0.5

head = {'Repo','Issue-no','Issue-title','Issue-body','Issue-reaction-total-count','Issue-reaction-plus-one','Issue-reaction-minus-one','Issue-comments'}
count = 0
for repo in repos_dict:
    while(int(repos_dict[repo])>0):
        row ={}
        time.sleep(sleep_time)
        r = requests.get(url = search_file_url.format(repo,repos_dict[repo]),headers=headers)

        # extracting data in json format
        issues = r.json()
        print(issues)
        if('message' in issues.keys()):
            print("Skipped Issue: ",int(repos_dict[repo]))
            repos_dict[repo] = int(repos_dict[repo]) - 1
            r = requests.get(url = search_file_url.format(repo,repos_dict[repo]),headers=headers)
            issues = r.json()
            print(issues)
        
        row['Repo'] = repo
        row['Issue-no'] = repos_dict[repo]
        if(issues['title'] is None):
            row['Issue-title'] = ''
        else:
            row['Issue-title'] = issues['title'].replace("/"," ")
        if(issues['body'] is None):
            row['Issue-body'] = ''
        else:
            row['Issue-body'] = issues['body'].replace("/"," ")
        row['Issue-reaction-total-count'] = issues['reactions']["total_count"]
        row['Issue-reaction-plus-one'] = issues['reactions']['+1']
        row['Issue-reaction-minus-one'] = issues['reactions'][ '-1']
        row['Issue-comments'] = issues['comments']
       
        
        filename="/drive/My Drive/Colab Notebooks/data/{}.{}".format(repo.replace("/","--")+"_raw", "csv")
        fileop = open(filename, 'a') #append existing file or create file with name <filename>
        
        csv_writer = csv.writer(fileop)

        if os.stat(filename).st_size == 0:
            print("Writing in", filename)
            csv_writer.writerow(row.keys())
            print(repo)
            
        csv_writer.writerow(row.values())

        print("Mined issue: ", repos_dict[repo])
        repos_dict[repo] = int(repos_dict[repo]) - 1
    fileop.close()
    print("Saved file: ",filename)